# Testing PHD MCMC

## Imports

In [ ]:
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
import corner
import phdmcmc

## Setting up

In [ ]:
np.random.seed(42)

# Choose the "true" parameters.
m_true = -0.9594
b_true = 4.294
f_true = 0.534

# Generate some synthetic data from the model.
N = 50
x = np.sort(10 * np.random.rand(N))
yerr = 0.1 + 0.5 * np.random.rand(N)
y = m_true * x + b_true
y += np.abs(f_true * y) * np.random.randn(N)
y += yerr * np.random.randn(N)

In [ ]:
def log_prior(theta, **kwargs):
    m, b, log_f = theta
    if -5.0 < m < 5.0 and -5.0 < b < 10.0 and -10.0 < log_f < 10.0:
        return 0.0, theta
    return -np.inf, theta

In [ ]:
def log_likelihood(theta, args=(), **kwargs):
    m, b, log_f = theta
    x, y, yerr = args
    model = m * x + b
    sigma2 = yerr**2 + model**2 * np.exp(2*log_f)
    return -0.5 * np.sum((y - model)**2 / sigma2)

# + np.log(sigma2)

In [ ]:
pos = (np.random.randn(100, 3) + 1) * np.array([-1, 4, 1])
nwalkers, ndim = pos.shape

print("nwalkers: {}".format(nwalkers))
print("ndim: {}".format(ndim))
print(pos[0])

## MCMC run

In [ ]:
%%time
full_acc, full_rej, full_probs, full_priors = \
    phdmcmc.mcmc_mh(log_likelihood, log_prior, pos, args=(x, y, yerr), stepsize=0.05,
                    nwalkers=nwalkers, iterations=50000, verbose=False)

In [ ]:
print len(full_acc[0])
print len(full_rej[0])

In [ ]:
%%time
acc, rej, probs, priors = phdmcmc.flat_chain(full_acc, full_rej, full_probs, full_priors, discard=0.1)
print acc.shape, rej.shape

In [ ]:
plt.plot(np.asarray(full_acc)[:, :, 0], "k", alpha=0.3)
plt.show()
plt.plot(np.asarray(full_acc)[:, :, 1], "k", alpha=0.3)
plt.show()
plt.plot(np.asarray(full_acc)[:, :, 2], "k", alpha=0.3)
plt.show()

In [ ]:
plt.hist2d(acc[:, 0], acc[:, 1], cmap='viridis', bins=40)
plt.plot(m_true, b_true, 'ro', ms=1.0)
plt.show()

# corner.corner(acc, color='#3F3D92', truths=[m_true, b_true, np.log(f_true)])

In [ ]:
m_mcmc = np.percentile(acc[:, 0], [16, 50, 84])
b_mcmc = np.percentile(acc[:, 1], [16, 50, 84])
logf_mcmc = np.percentile(acc[:, 2], [16, 50, 84])

print(m_mcmc, m_true)
print(b_mcmc, b_true)
print(logf_mcmc, f_true)